In [1]:
## Required Librarires
import pandas as pd
import numpy as np

In [13]:
## 2019 Survey Data
data  =  pd.read_excel(r'https://github.com/aiasci/HHBA/blob/main/raw/fert_cd/fert_cd_2019.xlsx?raw=true')

In [14]:
# TURKSTAT's Suggested Weights
ağırlık = []
for i in range(0, len(data)):
    if data['FERTNO'][i] == 1:
        ağırlık.append(data['FERTNO'][i])
    if data['FERTNO'][i] != 1:
        if data['YAS'][i] < 14:
            ağırlık.append(0.3)
    if data['FERTNO'][i] != 1:
        if data['YAS'][i] >= 14:
            ağırlık.append(0.5)
# Including Weights to the Survey Data
data['ağırlık'] = ağırlık

In [15]:
# Grouping Income by Households
hanehalkı_gelirler = data[['BIRIMNO', 'GELIR_TOPLAM','ağırlık']].groupby('BIRIMNO').sum() 
hanehalkı_gelirler = hanehalkı_gelirler.reset_index()
# Calculation of Equally Distributed Household Income According the Number of People That Constitutes The Household and Their Relative Weight 
hanehalkı_gelirler['Ortalama'] = hanehalkı_gelirler['GELIR_TOPLAM']/hanehalkı_gelirler['ağırlık']
# Sorting Dataset by Equally Distributed Household Income**** 
hanehalkı_gelirler.sort_values('Ortalama', inplace=True)
# **** TURKSTAT sorts households by their income which is a bit ambigious. Some household's contains more than 10 perosn that nearly half of them currently employed. 
# I think that TURKSTAT's methodology may require revision !!!

In [16]:
## If you want to continue with TURKSTAT's methodology please ignore the codeblock above
## Instead you may run the code below.
#hanehalkı_gelirler.sort_values('GELIR_TOPLAM', inplace=True)

In [17]:
# Re-indexing dataset
hanehalkı_gelirler =hanehalkı_gelirler.reset_index(drop=True)

In [18]:
# Creation of percentile Income Groups
hh_1 = hanehalkı_gelirler[:2305].reset_index(drop=True)
hh_1['Gelir Grubu'] = 1
hh_2 = hanehalkı_gelirler[2305:4609].reset_index(drop=True)
hh_2['Gelir Grubu'] = 2
hh_3 = hanehalkı_gelirler[4609:6913].reset_index(drop=True)
hh_3['Gelir Grubu'] = 3
hh_4 = hanehalkı_gelirler[6913:9217].reset_index(drop=True)
hh_4['Gelir Grubu'] = 4
hh_5 = hanehalkı_gelirler[9217:11521].reset_index(drop=True)
hh_5['Gelir Grubu'] = 5
# Aggregation of Income Groups
gruplar_main = pd.concat([hh_1, hh_2, hh_3, hh_4, hh_5])

In [19]:
## Enlistation of Households That Are in The Percentile Groups for Future Examination of Dataset
#a= pd.DataFrame()
#for i in range(1,6):
#    vars()['data_hh_'+str(i)] = pd.DataFrame()
#    for j in range(0,len(vars()['hh_'+str(i)])):
#        a= data[data['BIRIMNO'] == vars()['hh_'+str(i)]['BIRIMNO'][j]]
#        vars()['data_hh_'+str(i)] = vars()['data_hh_'+str(i)].append(a)
## Exporting Data to Excel
#for i in range(1,6):
#    vars()['data_hh_'+str(i)].to_excel(f'grup[{i}.xlsx',index=False) 

In [ ]:
# Household's Consumption Dataset
har = pd.read_csv(r'https://github.com/aiasci/HHBA/blob/main/raw/tuketim_cd/tuketim_cd_2019.csv?raw=true')
# Dataset of Consumption Bakset (Includes Names and HS5 codes)
madde = pd.read_excel(r'https://github.com/aiasci/HHBA/blob/main/raw/madde_tanim/hbs_madde_tanim2019.xlsx?raw=true')
# Renaming Column Names
col_names = ['ana_grup', 'HBS_KOD5','Grup adı']
madde.columns = col_names
# Loop for Turning HBs5 code to the integers
for j in range(0,len( madde)):
        madde['HBS_KOD5'][j] = madde['HBS_KOD5'][j].replace('.','')
        madde['HBS_KOD5'][j] = int(madde['HBS_KOD5'][j])
# Grouping Goods
a= []
tür = []
for j in range(0,len(madde)):
    if madde['HBS_KOD5'][j] < 3000:
        a = 'GIDA'
        tür.append(a)
    else:
        a = 'Diger'
        tür.append(a)
madde['tür'] = tür

In [ ]:
# Merging Consumption Dataset with Names of Goods by using HS5 codes
har_main = har.merge(madde, on = 'HBS_KOD5', how = 'inner')
# Merging Household's Consumption Dataset with Their Income Dataset by using Household Codes
df1 = har_main.merge(gruplar_main, on = 'BIRIMNO', how = 'inner')
# Aggregation of Household's Income considering their Income Group
gruplar_genel = gruplar_main[['GELIR_TOPLAM', 'Gelir Grubu']].groupby('Gelir Grubu').sum().reset_index()
# Narrowing Dataset According to Current Goal
df1 = df1[['BIRIMNO', 'HBS_KOD5', 'DEGER_D', 'tür', 'Grup adı', 'GELIR_TOPLAM', 'Gelir Grubu']]
# Aggregation of Household's Consumption by Goods considering Income Groups
ürün_gelirgrubu = df1[['HBS_KOD5', 'DEGER_D', 'Gelir Grubu']].groupby(['HBS_KOD5', 'Gelir Grubu']).sum().reset_index()
# Merging Aggregated Income and Consumption Datasets by using Income Groups
data_son = ürün_gelirgrubu.merge(gruplar_genel, on = 'Gelir Grubu', how = 'inner')
# Merging Names of the Goods
data_son = data_son.merge(madde[['HBS_KOD5', 'tür', 'Grup adı']], on = 'HBS_KOD5', how = 'inner')
# Calculation of How Much Each Income Groups Spend on Each Product
data_son['oran'] = data_son['DEGER_D']/(data_son['GELIR_TOPLAM']/12)

In [ ]:
# Expenditure for Food According to the Income Groups
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA')]['oran'].sum()
# Expenditure of a Certain Good to the Income Groups (By Using HB5 Code) 
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA') & (data_son['HBS_KOD5'] == 1111)]['oran'].sum()
# Expenditure of a Certain Good to the Income Groups (By Using Name of the Product) 
#data_son[(data_son['Gelir Grubu'] == 1) & (data_son['tür'] == 'GIDA') & (data_son['Grup adı'] == 'PİRİNÇ')]['oran'].sum()

In [ ]:
# Exporting results to the Excel
data_son.to_excel('data_son.xlsx', index=False)